In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tqdm
import shutil

In [2]:
def plot_data(input_df, which_plot: dict):
    plt.figure(figsize=(10, 6))
    if(which_plot["V"] == True):
        plt.plot(input_df['unix_ts'], input_df['V'], label='Voltage (V)')
    if(which_plot["I"] == True):
        plt.plot(input_df['unix_ts'], input_df['I'], label='Current (I)')
    if(which_plot["P"] == True):
        plt.plot(input_df['unix_ts'], input_df['P'], label='Power (P)')
    if(which_plot["Q"] == True):
        plt.plot(input_df['unix_ts'], input_df['Q'], label='Reactive Power (Q)')
    if(which_plot["S"] == True):
        plt.plot(input_df['unix_ts'], input_df['S'], label='Apparent Power (S)')

    plt.xlabel('Unix Timestamp')
    plt.ylabel('Values')
    plt.title(f'{which_plot["name"]}')
    plt.legend()
    plt.grid(True)
    plt.show()
    
def sort_filenames(filenames):
    """
    Sorts a list of filenames in ascending order based on the numeric part of the filename.

    Args:
        filenames (list): A list of filenames (e.g., ["fridge_1568.xlsx", "fridge_2773.xlsx", ...])

    Returns:
        list: Sorted list of filenames
    """
    def extract_numeric_part(filename):
        # Extract the numeric part from the filename (e.g., "fridge_1568.xlsx" -> 1568)
        return int(filename.split("_")[4].split(".")[0])

    # Sort the filenames based on the numeric part
    sorted_filenames = sorted(filenames, key=extract_numeric_part)

    return sorted_filenames

In [3]:
"lights_and_plugs_16_0.xlsx".split("_")[4].split(".")[0]

'0'

# Fridge auto label

# Unusual data threshold
1. LP20
- State 1: < 45W
- State 2: > 45W
2. LP3
- State 1: < 45W
- State 2: > 45W
3. LP16: 
- State 1: < 25W 
- State 2: > 25W
4. Fridge
- State 1: < 400W (~250W)
- State 2: > 400 W (~460W)
5. Clothes dryer: labeled by hand and remove transient.
- State 1: < 4000 W
- State 2: > 4000 W (~200W)
6. Oven: labeled by hand and remove transient.
- State 1: < 200W 
- State 2: 200 - 3000W
- State 3: > 3000W

In [5]:
# assign directory
directory = '/opt/nilm-shared-data/nilm_device_detection/other_dataset/RAE_dataset/transform_data/running_segments/lp20'
# out_dir = "../../data/RAE_dataset/transform_data/labeled_data/lp16"
unusual_files = []
# iterate over files in
# that directory
for filename in tqdm.tqdm(iterable=sort_filenames(os.listdir(directory))):
    fp = os.path.join(directory, filename)
    if os.path.isfile(fp):
        unlabeled_df = pd.read_excel(fp)
        # Drop 2 first rows, then the last row
        drop_idx = [0, 1, 2, len(unlabeled_df)-1, len(unlabeled_df)-2]
        labeled_df = unlabeled_df.drop(drop_idx)
        labeled_df.drop([labeled_df.columns[0], labeled_df.columns[1]], axis=1, inplace=True)
        power_np = labeled_df["P"].to_numpy()
        labeled_df["label"] = "lp16_s1"
        unexpected_values = power_np[power_np > 45]
        if len(unexpected_values) > 0:
            unusual_files.append(filename)
        # else:
            # labeled_df.to_excel(os.path.join(out_dir, filename), index=False)

100%|██████████| 372/372 [00:26<00:00, 14.00it/s]


In [6]:
print(len(unusual_files))

16


In [7]:
unusual_files

['lights_and_plugs_20_74.xlsx',
 'lights_and_plugs_20_127.xlsx',
 'lights_and_plugs_20_155.xlsx',
 'lights_and_plugs_20_157.xlsx',
 'lights_and_plugs_20_185.xlsx',
 'lights_and_plugs_20_187.xlsx',
 'lights_and_plugs_20_190.xlsx',
 'lights_and_plugs_20_218.xlsx',
 'lights_and_plugs_20_230.xlsx',
 'lights_and_plugs_20_231.xlsx',
 'lights_and_plugs_20_299.xlsx',
 'lights_and_plugs_20_306.xlsx',
 'lights_and_plugs_20_318.xlsx',
 'lights_and_plugs_20_347.xlsx',
 'lights_and_plugs_20_349.xlsx',
 'lights_and_plugs_20_353.xlsx']

In [ ]:
unusual_files

In [16]:
print(labeled_df.columns)
labeled_df.head()

Index(['unix_ts', 'Irms', 'pf', 'P', 'Q', 'S', 'label'], dtype='object')


,unix_ts,Irms,pf,P,Q,S,label
3,1510385787,0.3,0.5,18,22,36,lp16_s1
4,1510385788,0.3,0.5,18,22,36,lp16_s1
5,1510385789,0.3,0.5,18,22,36,lp16_s1
6,1510385790,0.3,0.5,18,22,36,lp16_s1
7,1510385791,0.3,0.5,18,22,36,lp16_s1


In [17]:
print(len(unusual_files))

2


In [21]:
for unusual_file in tqdm.tqdm(unusual_files):
    src_file = os.path.join(directory, unusual_file)
    des_file = os.path.join("/home/mrcong/Code/mylab-nilm-files/data_adapter/data/RAE_dataset/transform_data/labeled_data/lp16_need_label", unusual_file)
    unlabeled_df = pd.read_excel(src_file)
    # Drop 2 first rows, then the last row
    drop_idx = [0, 1, 2, len(unlabeled_df)-1, len(unlabeled_df)-2]
    labeled_df = unlabeled_df.drop(drop_idx)
    # Drop useless columns
    labeled_df.drop([labeled_df.columns[0], labeled_df.columns[1]], axis=1, inplace=True)
    labeled_df["label"] = "lp16_s"
    labeled_df.to_excel(des_file, index=False)

100%|██████████| 2/2 [00:00<00:00,  3.08it/s]
